# cli

> Fill in a module description here

In [ ]:
#| default_exp cli

In [ ]:
#| export
import json

from datasets import load_dataset
from fastcore.script import *
from hf_clean_benchmarks.core import BenchmarkCleaner

In [ ]:
#| hide
import logging

from nbdev.showdoc import *

# Turn off logging for datasets
logging.getLogger("datasets").setLevel(logging.ERROR)

In [ ]:
#| hide
import json
import tempfile

benchmarks = [
    {
        "name": "openai_humaneval",
        "splits": ["test"],
        "columns": ["prompt", "canonical_solution", "test"],
    },
    {
        "name": "mbpp",
        "splits": ["validation", "test"],
        "columns": ["text", "code", "test_list"],
    },
]

temp = tempfile.NamedTemporaryFile(prefix='pre_', suffix='_suf.json')
temp.write(json.dumps(benchmarks).encode())
print(temp.name)
temp.seek(0)
bm = json.load(temp)
print(bm)

/tmp/pre_c3y63zl4_suf.json
[{'name': 'openai_humaneval', 'splits': ['test'], 'columns': ['prompt', 'canonical_solution', 'test']}, {'name': 'mbpp', 'splits': ['validation', 'test'], 'columns': ['text', 'code', 'test_list']}]


In [ ]:
#| export
@call_parse
def clean_dataset(
    dataset_name: str, # Name of the dataset to clean
    column_name: str, # Name of the column to clean
    benchmark_configs_path: str, # Path to the benchmark configuration file
    output_path: str, # Path to where the cleaned dataset will be saved
    dataset_config_name: str = None, # Name of the dataset configuration to use
    data_dir: str = None, # Path to the data files to use
    dataset_split: str = "train", # Name of the dataset split to clean
    save_json: bool = False, # Whether to save the cleaned dataset as a JSON file
    ): 
    """
    Clean a dataset using a benchmark configuration file.
    """
    ds = load_dataset(
        dataset_name,
        name=dataset_config_name,
        data_dir=data_dir,
        split=dataset_split,
    )
    benchmarks = json.load(open(benchmark_configs_path))
    bench_cleaner = BenchmarkCleaner(benchmarks, threshold=0.1, num_perm=128)
    ds = bench_cleaner.clean(ds, column_name)
    if save_json:
        ds.to_json(output_path, orient="records", lines=True)
    else:
        ds.save_to_disk(output_path)

In [ ]:
#| exec_doc
clean_dataset(
    dataset_name="bigcode/the-stack-smol",
    column_name="content",
    benchmark_configs_path=temp.name,
    output_path="/tmp/test.jsonl",
    data_dir="data/python",
    dataset_split="train",
    save_json=True,
)

Checking for false positives...: 100%|██████████| 8674/8674 [01:07<00:00, 129.20it/s]


[11/06/22 10:39:25] INFO     Data Number                   : 10000                                      ]8;id=912292;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py\core.py]8;;\:]8;id=694905;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py#210\210]8;;\

                    INFO     Duplicate Number              : 4612                                       ]8;id=235532;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py\core.py]8;;\:]8;id=846901;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py#211\211]8;;\

                    INFO     Duplicate Rate                : 46.12%                                     ]8;id=982478;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py\core.py]8;;\:]8;id=643623;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py#212\212]8;;\

                    INFO     Total Time                    : 104.49 seconds                             ]8;id=792790;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py\core.py]8;;\:]8;id=828620;file:///home/nathan/projects/other/hf_clean_benchmarks/hf_clean_benchmarks/core.py#213\213]8;;\

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  1.77ba/s]


In [ ]:
#| hide
temp.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()